In [ ]:
import json 
import pandas as pd
import torch
import time
from sentence_transformers import SentenceTransformer, evaluation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# change paths depending on eli5 or wow 
data_folder = '/contextretrieval/bi-encoder/wow/splits/'

# output path stores csv file of results
output_path = '/contextretrieval/bi-encoder/wow/results/'

In [ ]:
def evaluator(model, test_passages, test_corpus, output_path):
    passages = dict(zip(test_passages['id'], test_passages['passages_text']))
    
    for index, row in test_passages.iterrows():
        row['relevant_ids'] = set(row['relevant_ids'])

    relevant_docs = dict(zip(test_passages['id'], test_passages['relevant_ids']))
    corpus = dict(zip(test_corpus['id'], test_corpus['input']))
    
    ir_evaluator = evaluation.InformationRetrievalEvaluator(passages, corpus, relevant_docs)
    
    return ir_evaluator(model, output_path=output_path)

In [ ]:
# load model 
bi_encoder = SentenceTransformer('msmarco-distilbert-base-tas-b') 

# read test data 
test_passages = pd.read_csv(data_folder + 'test_passages.csv', converters={'relevant_ids': pd.eval})
test_corpus = pd.read_csv(data_folder + 'test_corpus.csv')

In [ ]:
evaluator(bi_encoder, test_passages, test_corpus, output_path)